<a href="https://colab.research.google.com/github/sof1a03/KDE-group6/blob/main/book_cleaning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
import re
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
# A function to get the missing values count and it's percentage
def missing_values(df):
  """
  Description : This function takes a data frame as an input and gives missing value count and its percentage as an output
  function_name : missing_values
  Argument : dataframe.
  Return : dataframe

  """
  miss = df.isnull().sum() # finding the missing values.

  per = df.isnull().mean() # finding mean/ Average of missing values.
  df = pd.concat([miss,per*100],keys = ['Missing_Values','Percentage'], axis = 1) # concatenating both of them using concat method of pandas module.
  return df # returning dataframe

In [61]:
# Load datasets
books = pd.read_csv("Books.csv")

<ipython-input-61-c4f1c0ddb803>:2: DtypeWarning: Columns (3) have mixed types. Specify dtype option on import or set low_memory=False.
  books = pd.read_csv("Books.csv")


Books are identified by their respective ISBN. In order to not loose any book, any invalid ISBNs have already been identified and will be fixed in the dataset.
Note that in the case of several authors, only the first is provided.
Even if URLs linking to cover images are also given, appearing in three different flavors (Image-URL-S, Image-URL-M, Image-URL-L), i.e., small, medium, large, we're gonna drop this information because not relevant to our study case.

In [62]:
books.head()

,ISBN,Book_Title,Book_Author,Year_Of_Publication,Publisher,Image_URL_S,Image_URL_M,Image_URL_L
0,0195153448,Classical Mythology,Mark P. O. Morford,2002,Oxford University Press,http://images.amazon.com/images/P/0195153448.0...,http://images.amazon.com/images/P/0195153448.0...,http://images.amazon.com/images/P/0195153448.0...
1,0002005018,Clara Callan,Richard Bruce Wright,2001,HarperFlamingo Canada,http://images.amazon.com/images/P/0002005018.0...,http://images.amazon.com/images/P/0002005018.0...,http://images.amazon.com/images/P/0002005018.0...
2,0060973129,Decision in Normandy,Carlo D'Este,1991,HarperPerennial,http://images.amazon.com/images/P/0060973129.0...,http://images.amazon.com/images/P/0060973129.0...,http://images.amazon.com/images/P/0060973129.0...
3,0374157065,Flu: The Story of the Great Influenza Pandemic...,Gina Bari Kolata,1999,Farrar Straus Giroux,http://images.amazon.com/images/P/0374157065.0...,http://images.amazon.com/images/P/0374157065.0...,http://images.amazon.com/images/P/0374157065.0...
4,0393045218,The Mummies of Urumchi,E. J. W. Barber,1999,W. W. Norton &amp; Company,http://images.amazon.com/images/P/0393045218.0...,http://images.amazon.com/images/P/0393045218.0...,http://images.amazon.com/images/P/0393045218.0...


In [63]:
books.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 271360 entries, 0 to 271359
Data columns (total 8 columns):
 #   Column               Non-Null Count   Dtype 
---  ------               --------------   ----- 
 0   ISBN                 271360 non-null  object
 1   Book_Title           271360 non-null  object
 2   Book_Author          271358 non-null  object
 3   Year_Of_Publication  271360 non-null  object
 4   Publisher            271358 non-null  object
 5   Image_URL_S          271360 non-null  object
 6   Image_URL_M          271360 non-null  object
 7   Image_URL_L          271357 non-null  object
dtypes: object(8)
memory usage: 16.6+ MB


In [64]:
# Checking duplicates in datasets using duplicated method of dataframe.
print(f'''\t  Duplicates in books data is {books.duplicated().sum()}''')

	  Duplicates in books data is 0


In [65]:
books.describe()

,ISBN,Book_Title,Book_Author,Year_Of_Publication,Publisher,Image_URL_S,Image_URL_M,Image_URL_L
count,271360,271360,271358,271360,271358,271360,271360,271357
unique,271360,242135,102022,202,16807,271044,271044,271041
top,0195153448,Selected Poems,Agatha Christie,2002,Harlequin,http://images.amazon.com/images/P/185326119X.0...,http://images.amazon.com/images/P/185326119X.0...,http://images.amazon.com/images/P/225307649X.0...
freq,1,27,632,13903,7535,2,2,2


In [66]:
#correcting the ISBN
merged_invalid_books = pd.read_csv("merged_invalid_books.csv", encoding='latin1')

In [67]:
# Create a dictionary to map invalid ISBNs to their correct ISBN_true values
isbn_correction_map = dict(zip(merged_invalid_books['ISBN'], merged_invalid_books['ISBN_true']))
# Replace the ISBN values in books using the mapping
books['ISBN'] = books['ISBN'].replace(isbn_correction_map) #this is made in a separate ipynb code

In [68]:
# Drop unused columns
books = books.drop(columns=['Image_URL_S', 'Image_URL_M', 'Image_URL_L'], axis=1) # these columns are not relevant for our reccomendation system

In [69]:
missing_values(books)

,Missing_Values,Percentage
ISBN,0,0.000000
Book_Title,0,0.000000
Book_Author,2,0.000737
Year_Of_Publication,0,0.000000
Publisher,2,0.000737


number of missing values for Book_author and Publisher is negegable --> let's see if we can correct it

In [70]:
# Checking  for  null value in book author
books[books['Book_Author'].isna()]

,ISBN,Book_Title,Book_Author,Year_Of_Publication,Publisher
118033,0751352497,A+ Quiz Masters:01 Earth,NaN,1999,Dorling Kindersley
187689,9627982032,The Credit Suisse Guide to Managing Your Perso...,NaN,1995,Edinburgh Financial Publishing


both of books when we look online have no author, to simplify the process we're gonna drop these values.

In [71]:
books = books.dropna(subset=['Book_Author'])

In [72]:
books[books['Publisher'].isna()]

,ISBN,Book_Title,Book_Author,Year_Of_Publication,Publisher
128890,193169656X,Tyrant Moon,Elaine Corvidae,2002,NaN
129037,1931696993,Finders Keepers,Linnea Sinclair,2001,NaN


In [73]:
#Replacing NaNs with correct  values
books.loc[128890, 'Publisher'] = 'Mundania Press LLC'
books.loc[129037, 'Publisher'] = 'Bantam'

In [74]:
#insepcting the values in year of publication
books['Year_Of_Publication'].unique()

array([2002, 2001, 1991, 1999, 2000, 1993, 1996, 1988, 2004, 1998, 1994,
       2003, 1997, 1983, 1979, 1995, 1982, 1985, 1992, 1986, 1978, 1980,
       1952, 1987, 1990, 1981, 1989, 1984, 0, 1968, 1961, 1958, 1974,
       1976, 1971, 1977, 1975, 1965, 1941, 1970, 1962, 1973, 1972, 1960,
       1966, 1920, 1956, 1959, 1953, 1951, 1942, 1963, 1964, 1969, 1954,
       1950, 1967, 2005, 1957, 1940, 1937, 1955, 1946, 1936, 1930, 2011,
       1925, 1948, 1943, 1947, 1945, 1923, 2020, 1939, 1926, 1938, 2030,
       1911, 1904, 1949, 1932, 1928, 1929, 1927, 1931, 1914, 2050, 1934,
       1910, 1933, 1902, 1924, 1921, 1900, 2038, 2026, 1944, 1917, 1901,
       2010, 1908, 1906, 1935, 1806, 2021, '2000', '1995', '1999', '2004',
       '2003', '1990', '1994', '1986', '1989', '2002', '1981', '1993',
       '1983', '1982', '1976', '1991', '1977', '1998', '1992', '1996',
       '0', '1997', '2001', '1974', '1968', '1987', '1984', '1988',
       '1963', '1956', '1970', '1985', '1978', '1973', '1980'

In [75]:
# correcting DK publishing error
books[books['Year_Of_Publication'] == 'DK Publishing Inc']

,ISBN,Book_Title,Book_Author,Year_Of_Publication,Publisher
209538,078946697X,"DK Readers: Creating the X-Men, How It All Beg...",2000,DK Publishing Inc,http://images.amazon.com/images/P/078946697X.0...
221678,0789466953,"DK Readers: Creating the X-Men, How Comic Book...",2000,DK Publishing Inc,http://images.amazon.com/images/P/0789466953.0...


In [76]:
# on searching for these  books we came to know about its authors
#ISBN '078946697X'
books.loc[books.ISBN == '078946697X','Year_Of_Publication'] = 2000
books.loc[books.ISBN == '078946697X','Book_Author'] = "Michael Teitelbaum"
books.loc[books.ISBN == '078946697X','Publisher'] = "DK Publishing Inc"
books.loc[books.ISBN == '078946697X','Book_Title'] = "DK Readers: Creating the X - Men, How It All Began (Level 4: Proficient Readers)"

#ISBN '0789466953'
books.loc[books.ISBN == '0789466953','Year_Of_Publication'] = 2000
books.loc[books.ISBN == '0789466953','Book_Author'] = "James Buckley"
books.loc[books.ISBN == '0789466953','Publisher'] = "DK Publishing Inc"
books.loc[books.ISBN == '0789466953','Book_Title'] = "DK Readers: Creating the X - Men, How Comic Books Come to Life (Level 4: Proficient Readers)"

In [77]:
#checking the rows having 'Gallimard' as yearOfPublication
books.loc[books['Year_Of_Publication'] == 'Gallimard',:]

,ISBN,Book_Title,Book_Author,Year_Of_Publication,Publisher
220731,2070426769,"Peuple du ciel, suivi de 'Les Bergers\"";Jean-M...",2003,Gallimard,http://images.amazon.com/images/P/2070426769.0...


In [78]:
books.loc[books.ISBN=='2070426769','Year_Of_Publication']=2003
books.loc[books.ISBN=='2070426769','Book_Author']='Jean-Marie Gustave Le ClÃ?Â©zio'
books.loc[books.ISBN=='2070426769','Publisher']='Gallimard'
books.loc[books.ISBN=='2070426769','Book_Title']="Peuple du ciel, suivi de 'Les Bergers"

In [79]:
# changing dtype of year of publication
books['Year_Of_Publication'] =books['Year_Of_Publication'].astype(int)

In [80]:
# something is off about years of publication like:
books[(books['Year_Of_Publication'] > 0) & (books['Year_Of_Publication'] < 1800)]

,ISBN,Book_Title,Book_Author,Year_Of_Publication,Publisher
227531,9643112136,Dalan-i bihisht (Dastan-i Irani),Nazi Safavi,1378,Intisharat-i Quqnus
253750,964442011X,Tasht-i khun,IsmaÂ°il Fasih,1376,Nashr-i Alburz


In [81]:
#replacing with correct  values
books.loc[books.ISBN=='	9643112136','Year_Of_Publication'] = 2010
books.loc[books.ISBN=='964442011X', 'Year_Of_Publication'] = 1991

In [82]:
#Sustituting np.Nan in rows with year=0 or  greater than the current year,2024
books.loc[(books['Year_Of_Publication'] > 2024) | (books['Year_Of_Publication'] == 0),'Year_Of_Publication'] = np.NAN

# replacing NaN values with median value of Year_Of_Publication
books['Year_Of_Publication'].fillna(int(books['Year_Of_Publication'].median()), inplace=True)

<ipython-input-82-4a360287cf80>:5: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  books['Year_Of_Publication'].fillna(int(books['Year_Of_Publication'].median()), inplace=True)


In [83]:
books['Book_Author'].value_counts()

,count
Book_Author,
Agatha Christie,632
William Shakespeare,567
Stephen King,524
Ann M. Martin,423
Carolyn Keene,373
...,...
R. Gelman,1
Ellery R. Sheets,1
Artiste C. Arthur,1


In [84]:
books['Publisher'].value_counts()

,count
Publisher,
Harlequin,7535
Silhouette,4220
Pocket,3905
Ballantine Books,3783
Bantam Books,3646
...,...
Windstream Publishing Company,1
Polaris Books,1
Hannover House,1


In [85]:
books.info()

<class 'pandas.core.frame.DataFrame'>
Index: 271358 entries, 0 to 271359
Data columns (total 5 columns):
 #   Column               Non-Null Count   Dtype  
---  ------               --------------   -----  
 0   ISBN                 271358 non-null  object 
 1   Book_Title           271358 non-null  object 
 2   Book_Author          271358 non-null  object 
 3   Year_Of_Publication  271358 non-null  float64
 4   Publisher            271358 non-null  object 
dtypes: float64(1), object(4)
memory usage: 20.5+ MB


In [86]:
books = books.applymap(lambda x: x.lower() if isinstance(x, str) else x)
books[books.duplicated(subset=['Book_Title', 'Book_Author'], keep=False)]

<ipython-input-86-a09d4c2e54f8>:1: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  books = books.applymap(lambda x: x.lower() if isinstance(x, str) else x)


,ISBN,Book_Title,Book_Author,Year_Of_Publication,Publisher
0,0195153448,classical mythology,mark p. o. morford,2002.0,oxford university press
5,0399135782,the kitchen god's wife,amy tan,1991.0,putnam pub group
6,0425176428,what if?: the world's foremost military histor...,robert cowley,2000.0,berkley publishing group
7,0671870432,pleading guilty,scott turow,1993.0,audioworks
8,0679425608,under the black flag: the romance and the real...,david cordingly,1996.0,random house
...,...,...,...,...,...
271223,0451625145,thirteen days: a memoir of the cuban missile c...,robert f. kennedy,1988.0,signet book
271246,0195124995,for cause and comrades: why men fought in the ...,james m. mcpherson,1998.0,oxford university press
271316,0441011799,singularity sky,charles stross,2004.0,ace
271346,0395264707,dreamsnake,vonda n. mcintyre,1978.0,houghton mifflin


In [87]:
#Dropping the rows with the entire column values are duplicated
books.drop_duplicates(subset=['Book_Title', 'Book_Author'], keep='first', inplace=True)
books.reset_index(drop=True,inplace=True)

In [90]:
books[books['Book_Title']=='emma']

,ISBN,Book_Title,Book_Author,Year_Of_Publication,Publisher
1466,0451523067,emma,jane austen,1994.0,new amer library classics
157551,0553234315,emma,charlotte bronte,1983.0,bantam books (mm)


In [92]:
books.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 248238 entries, 0 to 248237
Data columns (total 5 columns):
 #   Column               Non-Null Count   Dtype  
---  ------               --------------   -----  
 0   ISBN                 248238 non-null  object 
 1   Book_Title           248238 non-null  object 
 2   Book_Author          248238 non-null  object 
 3   Year_Of_Publication  248238 non-null  float64
 4   Publisher            248238 non-null  object 
dtypes: float64(1), object(4)
memory usage: 9.5+ MB


In [93]:
books.to_csv('books_cleaned.csv', index=False)